In [ ]:
#MODELO PREDITIVO INDIVIDUAL RELATIVO AOS ALGORITMOS GENÉTICOS - MODELO DE REGRESSÃO DE AFINAÇÃO DOS PARÂMETROS --------------------------------------

import pandas as pd
import os
import random
import math
import numpy as np 
from matplotlib import pyplot as plt
import xlwings as xw
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from collections import Counter
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

#EXCEL - CAMINHO DO FICHEIRO
file_path = r"D:\Documentos\...\DataSet_Regressão_GA.xlsx"
dados_problema= pd.read_excel(file_path)
print (dados_problema)

    MH  NJOBS  NMACHINES  CMAXOPT OBJFUNC  CMAX  NUMGEN  TIMEEXEC
0   GA     10          5      655      WT   837     100     10.62
1   GA     10         10      887      WT  1504     100     13.50
2   GA     10         10      887      WT  1653     100     13.71
3   GA     15          5      890      WT  1020     100     27.41
4   GA     10         10      899      WT  1501     100      9.55
5   GA     10         10      899      WT  1560     100      9.66
6   GA     15          5      926      WT   987     100     24.70
7   GA     20          5     1165      WT  1624     100     69.50
8   GA     20         10     1218      WT  1627     100     52.48
9   GA     20          5     1222      WT  1222     100     56.67
10  GA     20         10     1424      WT  2310     100     54.03
11  GA     20         10     1424      WT  2421     100     58.11
12  GA     20         15     1446      WT  2406     100     56.01
13  GA     20         15     1446    Cmax  2482     100     54.43
14  GA    

In [11]:
#DIVIDIR DADOS EM TREINO E TESTE (RETIRAR COLONA CMAX, MH E TIMEEXEC (NÃO É UMA FEATURE))
dados_problema=dados_problema.drop(columns=['CMAX'])
dados_problema=dados_problema.drop(columns=['MH'])
dados_problema=dados_problema.drop(columns=['TIMEEXEC'])
#print (dados_problema)

X = dados_problema.drop(columns=['NUMGEN'])  #VARIÁVEIS INDEPENDENTES
y = dados_problema['NUMGEN'] #VARIÁVEL DEPENDENTE

#COLUNA OBJFUNX: PASSAR CMAX = 0 E WT=1 - NECESSÁRIO EM ALGUMAS TÉCNICAS DE MACHINE LEARNING
X['OBJFUNC'] = X['OBJFUNC'].map({'Cmax': 0, 'WT': 1})

#DIVIDIR OS DADOS EM CONJUNTO DE TREINO E TESTE 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#ÁRVORE DE DECISÃO -------------------------------------------------------------------------------------------------------------------------------------

#DEFINIR PARÂMETROS
param_grid = {
    'criterion': ['squared_error', 'absolute_error'], 
    'max_depth': [3, 5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 10, 15],
}

#CRIAR MODELO DE ÁRVORE DE DECISÃO
dt_regressor = DecisionTreeRegressor(random_state=42)

#GRIDSEAECHCV COM VALIDAÇÃO CRUZADA 
grid_search = GridSearchCV(
    estimator=dt_regressor, 
    param_grid=param_grid, 
    cv=5,  
    scoring='neg_mean_squared_error',
    n_jobs=-1 
)

#TREINAR O MODELO À PROCURA DOS MELHOR HIPERPARÂMETROS
grid_search.fit(X_train, y_train)

#OBTER O MELHOR MODELO
best_dt = grid_search.best_estimator_
print("Melhores Hiperparâmetros:", grid_search.best_params_)

#FAZER PREVISÕES
y_pred = best_dt.predict(X_test)

#CALCULAR MÉTRICAS DE AVALIAÇÃO
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

#RESULTADOS MÉTRICAS DE AVALIAÇÃO
print("\nMetricas de Avaliação:")
print(f"MSE (Mean Squared Error): {mse:.4f}")
print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")
print(f"MAE (Mean Absolute Error): {mae:.4f}")
print(f"MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
print(f"R² Score: {r2:.4f}")

#IMPORTÂNCIA DAS FEATURES
importances = best_dt.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print("\nImportância das Features:")
print(importance_df)


Melhores Hiperparâmetros: {'criterion': 'squared_error', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5}

Metricas de Avaliação:
MSE (Mean Squared Error): 14.3702
RMSE (Root Mean Squared Error): 3.7908
MAE (Mean Absolute Error): 2.7115
MAPE (Mean Absolute Percentage Error): 0.98%
R² Score: 0.9991

Importância das Features:
     Feature  Importance
0      NJOBS    0.999785
2    CMAXOPT    0.000151
1  NMACHINES    0.000037
3    OBJFUNC    0.000027


'\n#================ COMPARAÇÃO REAL VS PREVISTO ==================\nprint("\n=== Comparação dos 8 primeiros exemplos ===")\ncomparacao = pd.DataFrame({\'Real\': y_test[:8].values, \'Previsto\': y_pred[:8]})\ncomparacao = comparacao.round(4)\nprint(comparacao)\n\nfrom sklearn.tree import plot_tree\n\n# ====================== PLOT DA ÁRVORE DE DECISÃO ======================\nplt.figure(figsize=(20, 10))\nplot_tree(\n    best_dt, \n    feature_names=feature_names, \n    filled=True, \n    rounded=True, \n    fontsize=10,\n    max_depth=3  # Opcional: mostra até um certo nível da árvore\n)\nplt.title("Árvore de Decisão - Visualização")\nplt.tight_layout()\nplt.show()\n#plt.savefig("arvore_decisao.png", dpi=300)'

In [ ]:
#RANDOM FOREST -------------------------------------------------------------------------------------------------------------------------------------

#DEFINIR PARÂMETROS
param_grid = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_depth': [3, 5, 7, 10, 20, 30, None],  
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': [None, 'sqrt', 'log2'],
}

#CRIAR MODELO RANDOM FOREST
rf_regressor = RandomForestRegressor(random_state=42)

#GRID SEARCHCV COM VALIADAÇÃO CRUZADA
grid_search = GridSearchCV(
    estimator=rf_regressor, 
    param_grid=param_grid, 
    cv=5,  
    scoring='neg_mean_squared_error',
    n_jobs=-1  
)

#TREINAR O MODELO À PROCURA DOS MELHORES HIPERPARÂMETROS 
grid_search.fit(X_train, y_train)

#OBTER O MELHOR MODELO
best_rf = grid_search.best_estimator_
print("Melhores Hiperparâmetros:", grid_search.best_params_)

#PREVISÕES
y_pred = best_rf.predict(X_test)

#CALCULAR MÉTRICAS DE AVALIAÇÃO
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

#RESULTADOS 
print("\nMetricas de Avaliação:")
print(f"MSE (Mean Squared Error): {mse:.4f}")
print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")
print(f"MAE (Mean Absolute Error): {mae:.4f}")
print(f"MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
print(f"R² Score: {r2:.4f}")

#================ IMPORTÂNCIA DAS FEATURES ==================
importances = best_rf.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})
importance_df['Importance'] = importance_df['Importance'].round(4)
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print("\nImportância das Features:")
print(importance_df)


Melhores Hiperparâmetros: {'max_depth': 5, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

Metricas de Avaliação:
MSE (Mean Squared Error): 138.5621
RMSE (Root Mean Squared Error): 11.7712
MAE (Mean Absolute Error): 5.3146
MAPE (Mean Absolute Percentage Error): 3.96%
R² Score: 0.9910

Importância das Features:
     Feature  Importance
0      NJOBS      0.5198
2    CMAXOPT      0.4801
3    OBJFUNC      0.0001
1  NMACHINES      0.0000


'\n# Plot\nplt.figure(figsize=(10, 6))\nsns.barplot(x=\'Importance\', y=\'Feature\', data=importance_df, palette=\'viridis\')\nplt.title(\'Importância das Features - Decision Tree\')\nplt.tight_layout()\nplt.show()\n\n#================ COMPARAÇÃO REAL VS PREVISTO ==================\nprint("\n=== Comparação dos 8 primeiros exemplos ===")\ncomparacao = pd.DataFrame({\'Real\': y_test[:8].values, \'Previsto\': y_pred[:8]})\ncomparacao = comparacao.round(4)\nprint(comparacao)'

In [ ]:
#REGRESSÃO LINEAR -------------------------------------------------------------------------------------------------------------------------------------

#MODELO DE REGRESSÃO LINEAR
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

#PREVISÕES
y_train_pred = lr_model.predict(X_train)
y_test_pred = lr_model.predict(X_test)

#CÁLCULO DAS MÉTRICAS DE AVALIAÇÃO
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mape = mean_absolute_percentage_error(y_test, y_test_pred) * 100
test_r2 = r2_score(y_test, y_test_pred)

# RESULTADOS
print("\nMÉTRICAS DE AVALIAÇÃO:")
print(f"MSE: {test_mse:.4f}")
print(f"RMSE: {test_rmse:.4f}")
print(f"MAE: {test_mae:.4f}")
print(f"MAPE: {test_mape:.4f}%")
print(f"R²: {test_r2:.4f}")



MÉTRICAS DE AVALIAÇÃO:
MSE: 421.5533
RMSE: 20.5318
MAE: 16.4200
MAPE: 14.2170%
R²: 0.9727
